steps:
- recalssify rasters into groups based on bins or stats
- convert the reclassified rasters to vectors
- reclassification using vectors
- use Kmeans as first pass with minimum area for a class (to avoid tiny blobs)
- use sklearn image classifiers to shorthand the work


Note on stratification - this POC does not include the step fo first excluding areas with 20% or greater tree cover (which cannot be included for carbon projects). This POC just builds out the basic backbone of notebooks that allow us to input several rasters and person some terrain analysis, geometric operations and data reclassification to get a series of polygons representing 'similar areas' within the procided area of interest. 

In [2]:
%pip install scikit-learn -q
%pip install spectral -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import geopandas as gpd
import rasterio as rst

from sklearn.cluster import KMeans
